In [1]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client

c = Client()


/home/alberta/anaconda2/envs/pangeo-cloud/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
c

Client Scheduler: tcp://127.0.0.1:42257 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.66 GB


In [3]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
import numpy.ma as ma

today=date.today()



In [4]:
data_dir = '/store/molines/NATL60/NATL60-CJM165-S/1h/'
gridfile='/store/molines/NATL60/NATL60-I/NATL60_v4.1_cdf_mesh_hgr.nc'
dsgrid=xr.open_dataset(gridfile,chunks={'x':1000,'y':1000})

In [ ]:
def dx_var(data,e1):
    dx_var = (data.shift(x=-1) - data)/e1
    return dx_var
def dy_var(data,e2):
    dy_var = (data.shift(x=-1) - data)/e2
    return dy_var
    

In [ ]:
filetyps = {'votemper' : 'gridT', 'vosaline' : 'gridS'}
filedeps = {'votemper' : 'deptht','vosaline' : 'deptht'}
filee1 = {'votemper' : 'e1t','vosaline' : 'e1t'}

date='20121212' 
year=date[0:4]
month=date[4:6]
day=date[6:]
filename = sorted(glob.glob(data_dir+'/'+str(year)+'/NATL60-CJM165_y'+str(year)+'m'+str(month)+'d'+str(day)+'.1h_gridT.nc'))
file=filename[0]
ds=xr.open_dataset(file,chunks={'x':1000,'y':1000,'time_counter':1})
dataT=ds['sosstsst']
dataS=ds['sosaline']
e1=dsgrid['e1t'][0]
e2=dsgrid['e2t'][0]
dataT_dx=dx_var(dataT,e1)
dataS_dx=dx_var(dataS,e1)
dataT_dy=dy_var(dataT,e1)
dataS_dy=dy_var(dataS,e1)
R=(dataT_dx + 1j * dataT_dy)/(dataS_dx + 1j * dataS_dy)
magR=np.abs(R)
magR_ma=ma.masked_invalid(magR)
Tu=np.arctan(magR)

In [ ]:
!pip install windrose 


In [ ]:
from windrose import WindroseAxes

fig = plt.figure(figsize=(10, 5))

left_rectangle = [0, 0.1, 0.4, 0.75]     # [left, bottom, width, height] as a fraction of total figure size
ax1 = fig.add_axes(left_rectangle)    # creates the axes of specified dimensions

mto[wsp].hist(bins=100)
ax1.set_xscale("log")
ax1.set_xlabel("wind speed (m/s)")

right_rectangle = [0.5, 0.1, 0.5, 0.75]     # [left, bottom, width, height]
ax = WindroseAxes(fig, right_rectangle)
fig.add_axes(ax)
ax.bar(mto[wdir], mto[wsp], normed=True, opening=0.8, edgecolor='white', bins=np.logspace(-1,1, 10))
ax.set_title("annual", position=(0.5, 1.1))

ax.set_legend()
ax.legend(title="wind speed (m/s)", loc=(1.1, 0))

In [ ]:
def compute_turner_angle(date):
    year=date[0:4]
    month=date[4:6]
    day=date[6:]
    filename = sorted(glob.glob(data_dir+'/'+str(year)+'/NATL60-CJM165_y'+str(year)+'m'+str(month)+'d'+str(day)+'.1h_gridT.nc'))
    file=filename[0]
    ds=xr.open_dataset(file,chunks={'x':1000,'y':1000,'time_counter':1})
    dataT=ds['sosstsst']
    dataS=ds['sosaline']
    e1=dsgrid['e1t'][0]
    e2=dsgrid['e2t'][0]
    dataT_dx=dx_var(dataT,e1)
    dataS_dx=dx_var(dataS,e1)
    dataT_dy=dy_var(dataT,e1)
    dataS_dy=dy_var(dataS,e1)
    R=(dataT_dx + 1j * dataT_dy)/(dataS_dx + 1j * dataS_dy)
    magR=np.abs(R)
    magR_ma=ma.masked_invalid(magR)
    Tu=np/arctan(R)